In [ ]:
import numpy as np
from copy import deepcopy
import json
import os
from typing import List, Tuple
import uuid
import re
import pandas as pd
from IPython.display import display

## Utilidades

In [ ]:
def plural(palavra: str) -> str:
    """
    Passa uma palavra em português para o plural. Não funciona sempre,
    devido à quantidade de exceções que precisam ser programadas à mão.
    """
    palavra = palavra.strip()
    if len(palavra.split(" ")) > 1:
        return palavra
    if not palavra:
        return palavra
    invariaveis = [r"x$"]
    for p in invariaveis:
        if re.search(p, palavra):
            return palavra
    substituicoes = {
        r"ão": r"õe",
        r"r$": r"re",
        r"z$": r"ze",
        r"s$": r"se",
        r"(?<=[aeou])l": r"i",
        r"il": r"ei",
        r"m$": r"n",
    }
    for p, s in substituicoes.items():
        palavra = re.sub(p, s, palavra)
    return palavra + "s" if palavra[-1] != "s" else palavra

def sanitize(palavra: str) -> str:
    """Substitui caracteres acentuados."""
    substituicoes = {
        "a": ["á", "â", "ã", "à"],
        "c": ["ç"],
        "e": ["é", "ê"],
        "i": ["í"],
        "o": ["ó", "ô", "õ"],
        "u": ["ú", "ü"],
    }
    for substituta, letras in substituicoes.items():
        for letra in letras:
            palavra = palavra.replace(letra, substituta)
    return palavra

In [ ]:
def drop_duplicates(ls: list) -> list:
    """Remove elementos duplicados ou vazios."""
    return list(filter(None, set(ls)))

def flatten(ls: list) -> list:
    """Reduz em 1 a dimensão de uma lista."""
    return [item for sublist in ls for item in sublist]

def remove(ls: list, to_remove: list) -> list:
    """Remove de uma lista quaisquer elementos que estejam em outra."""
    return [item for item in ls if item not in to_remove]

def is_flat(x) -> bool:
    """Retorna verdadeiro se a entrada não for um dicionário nem uma lista."""
    return not (isinstance(x, list) or isinstance(x, dict))

def is_flat_list(x) -> bool:
    """Retorna verdadeiro se a lista for plana."""
    return all(is_flat(y) for y in x)

def mix_list(a, b) -> list:
    """
    Realiza um outer join entre duas listas planas ou que contêm dicionários.
    """
    if is_flat_list(a) and is_flat_list(b):
        return drop_duplicates(a + b)
    elif is_flat_list(a) or is_flat_list(b):
        raise ValueError(a, b)

    a_dict, a_key = to_dict(a)
    b_dict, b_key = to_dict(b)
    if a_key != b_key:
        raise ValueError(f"A different global key was found for each list: {a_key} "
                         f"and {b_key}, respectively")
    mixed_dicts = mix_dict(a_dict, b_dict)
    return list(mixed_dicts.values())

def to_dict(ls: List[dict]) -> Tuple[dict, str]:
    """
    Transforma uma lista de dicionários em um dicionário de dicionários, 
    inferindo como chave para cada dict algum valor dele que seja presente
    em todos os dicts, único em cada um e plano.
    """
    if len(ls) == 1:
        return {"placeholder_key": ls[0]}, "placeholder_key"
    d = ls[0]
    global_keys = []
    hardcoded_precedence = {"conditions": 1, "title": 0}
    for key in d:
        # se o valor dessa chave não for plano
        if not is_flat(d[key]):
            # print(key, "is not flat")
            continue
        # se nem todos dicts tiverem essa chave
        if not all(key in d_ for d_ in ls):
            # print(key, "is not in every dict")
            continue
        # se nem todos os dicts tiverem um valor para essa chave
        if not all(bool(d_[key]) for d_ in ls):
            # print(key, "doesnt have a value in all dicts")
            continue
        # se cada dict não tiver um valor único para essa chave
        if not len(set(d_[key] for d_ in ls)) == len(ls):
            # print(key, "doesnt have all unique values")
            continue
        global_keys.append(key)

    if not global_keys:
        raise ValueError(f"Could not find a global key for {ls}")
    global_keys.sort(key=lambda x: hardcoded_precedence.get(x, 999))
    global_key = global_keys[0]
    return {d_[global_key]: d_ for d_ in ls}, global_key

def mix_dict(a: dict, b: dict) -> dict:
    """
    Retorna um dicionário contendo as chaves e valores de ambos dicionários
    de entrada, dando preferência para os valores de b.
    """
    out = deepcopy(b)
    for k, v in a.items():
        if k not in out:
            out[k] = v
        elif isinstance(v, list):
            if not isinstance(b[k], list):
                raise ValueError(f"a é uma lista em {k}, mas b é {type(b[k])}")
            out[k] = mix_list(v, b[k])
        elif isinstance(v, dict):
            if not isinstance(b[k], dict):
                raise ValueError(f"a é um dict em {k}, mas b é {type(b[k])}")
            out[k] = mix_dict(v, b[k])
        else:
            out[k] = v
    return out

def remove_nans(d: dict) -> dict:
    """Removes keys with nan value from a dict"""
    return {k : v for k, v in d.items() if v not in [np.nan]}

## I/O

In [ ]:
def load_questions() -> pd.DataFrame:
    df = pd.read_excel("results/Perguntas.xlsx", sheet_name="finais")
    df = df.dropna(subset=["Resposta"])
    subs = {
        "Pergunta": "pergunta",
        "Resposta": "resposta",
        "Intenção": "intent",
        "Rótulos": "rótulos",
        "Modificador": "modificador",
        "Substantivo": "substantivo",
        "Recipiente": "recipiente",
        "Elocuções": "examples",
    }
    df = df[list(subs.keys())]
    df = df.rename(columns=subs)
    df = df.fillna("")
    return df

def load_skill(file_path: str) -> dict:
    with open(file_path, "r", encoding="utf-8") as f:
        sk = json.load(f)
    return sk

def save_skill(file_path: str, to_save: dict):
    root, extension = os.path.splitext(file_path)
    new_skill_path = f"{root}2{extension}"
    with open(new_skill_path, "w", encoding="utf-8") as f:
        json.dump(to_save, f, ensure_ascii=False)
    print(f"Skill saved as {new_skill_path}!")

## Intenções

In [ ]:
def get_intents(df: pd.DataFrame) -> dict:
    subset = ["intent", "pergunta", "examples"]
    records = df[subset].to_dict(orient="records")
    intents = [
        {
            "intent": record["intent"],
            "examples": get_examples(record),
            "description": "",
        }
        for record in records
    ]
    intents.sort(key=lambda x: x["intent"])
    return intents

def get_examples(record: dict) -> List:
    # a própria pergunta é um exemplo
    out = [{"text": record["pergunta"]}]
    # tudo que está em Elocuções é exemplo também
    if record["examples"]:
        out += [{"text": exemplo} for exemplo in record["examples"].split("--")]
    return out

## Entidades

In [ ]:
def get_entities(df):
    subset = ["rótulos", "modificador", "substantivo", "recipiente"]
    entities = [
        {"entity": col, "values": get_entity_values(df[col]), "fuzzy_match": True}
        for col in subset
    ]
    entities.sort(key=lambda x: x["entity"])
    return entities

def get_entity_values(series):
    records = series.drop_duplicates().to_list()
    records = [r.split("-") for r in records]
    records = flatten(records)
    records = drop_duplicates(records)
    values = [
        {"type": "synonyms", "value": record, "synonyms": []} for record in records
    ]
    return values

## Nós de diálogo

In [ ]:
def get_dialog_nodes(df: pd.DataFrame) -> dict:
    records = df.to_dict(orient="records")
    nodes = [
        {
            "type": "standard",
            "title": get_titulo(record),
            "output": {
                "generic": [
                    {
                        "values": [{"text": record["resposta"]}],
                        "response_type": "text",
                        "selection_policy": "sequential",
                    }
                ]
            },
            "context": {"contexto": sanitize(record["rótulos"])},
            "conditions": get_all_conditions(record),
            "dialog_node": f"node_{uuid.uuid4().hex[:16]}"
        }
        for record in records
    ]
    nodes.sort(key=lambda x: x["conditions"])
    return nodes

def get_titulo(js: dict) -> str:
    """
    Retorna um título para o nó baseado no modificador, substantivo, recipiente
    e rótulos.
    :param js: nó
    :return: título
    """
    modificador = js["modificador"]
    substantivo = js["substantivo"].replace("-", " ")
    recipiente = js["recipiente"].replace("-", " ")
    
    contextos = get_contextos(js["rótulos"].split("_"))
    contextos = list(map(lambda x: x.replace("-", " "), contextos))
    trechos = []
    if contextos:
        trechos.append(f"{'/'.join(contextos)}:")
    
    if modificador in ["efeito"]:
        trechos.append(modificador)
        if substantivo:
            trechos.append(f"de {substantivo}")
        recipiente = recipiente or contextos[0]
        trechos.append(f"em {recipiente}")
    elif modificador in ["definição"]:
        trechos.append(modificador)
        if substantivo and recipiente:
            raise ValueError(f"Perguntas do tipo 'definição' não podem ter substantivo e recipiente, "
                             "apenas um deles! Pergunta: {js['pergunta']}")
        elif substantivo or recipiente:
            trechos.append(f"de {substantivo or recipiente}")
    elif modificador in ["maiores", "menores"]:
        substituir = {"produção": "produtores"}
        if recipiente in substituir:
            recipiente = substituir[recipiente]
        substantivo = plural(substantivo)
        if recipiente:
            trechos.append(f"{modificador} {substantivo} {recipiente}?")
        else:
            trechos.append(f"{modificador} {substantivo}?")
    elif modificador in ["diferença"]:
        recipiente = recipiente or contextos[0]
        trechos.append(f"{modificador} entre {substantivo} e {recipiente}?")
    elif modificador in ["existe", "quantidade"]:
        recipiente = recipiente or "no Brasil"
        trechos.append(modificador)
        if substantivo:
            trechos.append(substantivo)
        trechos.append(f"{recipiente}?")
    elif modificador in ["listar"]:
        substituir = {"extinção": "em extinção", "aaz": "na Amazônia Azul", "brasil": "no Brasil"}
        if recipiente in substituir:
            recipiente = substituir[recipiente]
        substantivo = plural(substantivo)
        trechos.append(f"exemplos de {substantivo} {recipiente}")
    elif modificador in ["pertence"]:
        if substantivo and recipiente:
            trechos.append(f"{substantivo} é um {recipiente}?")
        elif substantivo:
            trechos.append(f"{substantivo} é um {contextos[0]}?")
        elif recipiente:
            trechos.append(f"{contextos[0]} é um {recipiente}?")
        else:
            raise ValueError(f"Perguntas do tipo 'pertence' precisam de substantivo ou de recipiente! Pergunta: {js['pergunta']}")
    else:
        trechos.append(modificador)
        if substantivo:
            trechos.append(substantivo)
        if recipiente:
            trechos.append(recipiente)
    trechos[0] = trechos[0].capitalize()
    titulo = " ".join(trechos)
    titulo = titulo.strip()
    return titulo

def get_contextos(rotulos):
    """
    Devolve os contextos de uma pergunta baseado em seus rótulos,
    sendo que há uma lista de rótulos que não definem contexto.
    """
    rotulos_nao_contextuais = [
        "fauna",
        "flora",
        "outras",
        "física",
        "símbolo",
        "turismo",
        "engenharia",
        "saúde",
        "geologia",
    ]
    contextos = [contexto for contexto in rotulos if all(rot not in contexto for rot in rotulos_nao_contextuais)]
    return contextos

def get_all_conditions(js):
    modificador = js["modificador"]
    substantivo = js["substantivo"]
    recipiente = js["recipiente"]
    rotulos = js["rótulos"].split("-") + [js["rótulos"]]
    rotulos = drop_duplicates(rotulos)
    contextos = get_contextos(rotulos)

    if contextos:
        conds_adicionais = [get_single_condition(modificador, substantivo, recipiente, contexto) for contexto in contextos]
    else:
        conds_adicionais = [get_single_condition(modificador, substantivo, recipiente)]

    conds = [f"#{js['intent']}"] + flatten(conds_adicionais)
    cond_str = " || ".join(conds)
    return cond_str

def get_single_condition(modificador, substantivo, recipiente, contexto=None) -> list:
    trechos = [f"@modificador:{modificador}"]
    if substantivo:
        trechos.append(f"&& @substantivo:{substantivo}")
    if recipiente:
        trechos.append(f"&& @recipiente:{recipiente}")
    cond = " ".join(trechos)
    if contexto:
        contexto = sanitize(contexto)
        out = [cond + f' && $contexto=="{contexto}"', cond + f" && @rótulos:{contexto}"]
    else:
        out = [cond]
    return out

def convert_to_list(df: pd.DataFrame) -> List[dict]:
    list_of_dicts = df.to_dict(orient="records")
    return [remove_nans(d) for d in list_of_dicts]

In [ ]:
class NodeOrganizer:
    """
    Assumes nodes have either been added via the interface (manual) or generated automatically
    by this script (generated). There is one special case, the last node (anything_else),
    which was added via interface but must always be the last one.
    """
    def __init__(self, nodes: List[dict]):
        self.df = pd.DataFrame(nodes)
        self._separate_nodes()

    def _separate_nodes(self):
        self.anything_else_node = (self.df.conditions == "anything_else") & (self.df.parent.isna())
        self.generated_nodes = ~self.df.dialog_node.str.match(r"node_._")
        self.manual_nodes = ~self.generated_nodes & ~self.anything_else_node
        self.df_anything_else = self.df[self.anything_else_node].copy()
        self.df_generated = self.df[self.generated_nodes].copy()
        self.df_manual = self.df[self.manual_nodes].copy()
        self.root = self.df_manual.previous_sibling.isna() & self.df_manual.next_step.isna()
        self.root_folder = self.root & (self.df_manual.type == "folder")

    def _build(self):
        self.df = self.df_manual.append(self.df_generated).append(self.df_anything_else)
        self.df.reset_index(drop=True, inplace=True)
        self._separate_nodes()

    def run(self, intent_limit: int = 0):
        self.sort_nodes()
        self.limit_intents(intent_limit)
        self.cleanup_previous_siblings()
        self.fix_previous_siblings()

    def sort_nodes(self):
        self.sort_manual_nodes_by_previous_siblings()
        self.df_generated.sort_values(by=["conditions"])
        self._build()
        print("Nodes sorted!")

    def sort_manual_nodes_by_previous_siblings(self):
        df = self.df_manual
        previous_siblings = df.previous_sibling.to_list()
        parents = df.parent.to_list()
        root = df.previous_sibling.isna() & df.next_step.isna()
        root_folder = root & (df.type == "folder")
        root_node = root & ~root_folder
        df.loc[root_folder, "order"] = 0
        df.loc[root_node, "order"] = 1

        curr = 2
        last_node = df[root_node].dialog_node.values[0]
        last_parent = ""
        while df.order.hasnans:
            if last_node in parents:
                next_node = (df.parent == last_node) & (df.previous_sibling.isna())
                last_parent = last_node
            elif last_node in previous_siblings:
                next_node = df.previous_sibling == last_node
            else:
                next_node = df.previous_sibling == last_parent
            df.loc[next_node, "order"] = curr
            try:
                last_node = df[next_node].dialog_node.values[0]
            except IndexError:
                break
            curr += 1

        df.sort_values(by=["order"], inplace=True)
        df.drop("order", inplace=True, axis=1)

    def cleanup_previous_siblings(self):
        """
        Removes the previous_sibling field of a node when it references a non-existent
        node identifier.
        """
        self.df.loc[self.generated_nodes | self.anything_else_node, "previous_sibling"] = np.nan
        self._separate_nodes()
        print("Previous siblings cleaned up!")

    def fix_previous_siblings(self):
        """
        Applies previous_sibling to nodes which don't have one (generated) based on the
        node above. Then, connects the generated nodes to the manual nodes and the last
        node (anything_else).
        """
        no_upstream = self.df_generated.previous_sibling.isna() & self.df_generated.parent.isna()
        df_no_upstream = self.df_generated[no_upstream].copy()
        df_no_upstream.reset_index(inplace=True)
        df_no_upstream.previous_sibling = df_no_upstream.dialog_node.shift(1)

        # assign the last manual node as the previous sibling for the first generated node
        last_manual_node = self.df_manual[self.root_folder].dialog_node[0]
        df_no_upstream.loc[0, "previous_sibling"] = last_manual_node
        df_no_upstream.set_index("index", inplace=True)

        # assign the last generated node as the previous sibling for the anything_else node
        last_generated_node = self.df_generated.dialog_node.to_list()[-1]
        self.df.loc[self.anything_else_node, "previous_sibling"] = last_generated_node

        # overwrites self.df with values from df_no_upstream based on index
        self.df.update(df_no_upstream)
        print("Previous siblings fixed!")

    def limit_intents(self, n: int):
        if not n:
            return
        manual_intents = self.df_manual.conditions.str.contains(r"#\S+")
        self.df_generated = self.df_generated[:n - manual_intents.sum()]
        self._build()
        print("Intents limited!")

    def get_intents(self)-> List[dict]:
        df = self.df.copy()
        df["intent"] = df.conditions.astype(str).apply(
            lambda x: re.search(r"#(\S+)", x).group(1) if re.search(r"#(\S+)", x) else ""
        )
        intents = df.intent.to_list()
        return drop_duplicates(intents)

## Skill

In [ ]:
def mix_skills(base, **kwargs):
    new = deepcopy(base)
    for k, v in kwargs.items():
        new[k] = v
    return new

## Rodar

In [ ]:
questions = load_questions()
questions[~(questions["examples"]=="")]

In [ ]:
skill_path = "results/skill-Amazônia-Azul.json"
skill = load_skill(skill_path)

In [ ]:
new_intents = get_intents(questions)
print("Intents obtained!")
mixed_intents = mix_list(skill["intents"], new_intents)
print("Intents mixed!")

In [ ]:
new_entities = get_entities(questions)
print("Entities obtained!")
mixed_entities = mix_list(skill["entities"], new_entities)
print("Entities mixed!")

In [ ]:
new_nodes = get_dialog_nodes(questions)
print("Nodes obtained!")
mixed_nodes = mix_list(skill["dialog_nodes"], new_nodes)
print("Nodes mixed!")

In [ ]:
node_organizer = NodeOrganizer(mixed_nodes)
node_organizer.run(intent_limit=100)

In [ ]:
used_intents = node_organizer.get_intents()
mixed_intents = [intent for intent in mixed_intents if intent["intent"] in used_intents]


In [ ]:
mixed_nodes = convert_to_list(node_organizer.df)
mixed_skill = mix_skills(skill, intents=mixed_intents, entities=mixed_entities, dialog_nodes=mixed_nodes)
save_skill(skill_path, mixed_skill)